# shortify Analysis
短縮URLサービスのエンドツーエンドテストと評価をこのNotebookで実施します。

## 0. 事前準備
1. `docker compose up -d` で `web` `redis` `jupyter` を起動する
2. Notebookコンテナ内から `http://web:8000` でFastAPIにアクセスする
3. 必要に応じて `SHORTIFY_API_BASE` 環境変数でAPIエンドポイントを上書きする

In [ ]:
import os
import statistics
import time

import requests

BASE_URL = os.getenv("SHORTIFY_API_BASE", "http://web:8000").rstrip('/')
print(f'Using API endpoint: {BASE_URL}')

In [ ]:
def shorten_url(long_url: str) -> dict:
    response = requests.post(
        f'{BASE_URL}/api/v1/data/shorten',
        json={'url': long_url},
        timeout=5,
    )
    response.raise_for_status()
    return response.json()


def fetch_redirect(short_id: str) -> requests.Response:
    response = requests.get(
        f'{BASE_URL}/{short_id}',
        allow_redirects=False,
        timeout=5,
    )
    return response


def exercise_shortening(sample_url: str) -> tuple[dict, requests.Response]:
    result = shorten_url(sample_url)
    redirect_response = fetch_redirect(result['slug'])
    return result, redirect_response

In [ ]:
sample = 'https://example.com/articles/shortify-design?ref=notebook'
shortened, redirect_response = exercise_shortening(sample)
shortened

In [ ]:
redirect_response.status_code, redirect_response.headers.get('Location')

In [ ]:
def bulk_generate(n: int = 1000) -> dict:
    start = time.perf_counter()
    slugs = []
    for idx in range(n):
        long_url = f'https://example.com/resource/{idx}?ts={time.time_ns()}'
        result = shorten_url(long_url)
        slugs.append(result['slug'])
    elapsed = time.perf_counter() - start
    unique_count = len(set(slugs))
    return {
        'total': n,
        'elapsed_seconds': elapsed,
        'per_second': n / elapsed if elapsed else float('inf'),
        'unique_slugs': unique_count,
        'has_collisions': unique_count != n,
    }

bulk_metrics = bulk_generate()
bulk_metrics

In [ ]:
assert bulk_metrics['elapsed_seconds'] < 1, 'Generation exceeded 1 second'
assert not bulk_metrics['has_collisions'], 'Duplicate short IDs detected'
bulk_metrics

In [ ]:
# Redisキャッシュによる再取得の即時性を測定
single = shorten_url('https://example.com/cache-check')
slug = single['slug']

# 1回目の取得（ウォームアップ）
fetch_redirect(slug)

latencies: list[float] = []
for _ in range(5):
    start = time.perf_counter()
    response = fetch_redirect(slug)
    response.raise_for_status()
    latencies.append(time.perf_counter() - start)

{"latencies": latencies, "mean_ms": statistics.mean(latencies) * 1000}